In [1]:
import pandas as pd
from io import StringIO
import requests

url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTSLyp3Ev2bxe8pnHStFtbdwg-EhvbMEhEUzRsaFjC0sKVQ-OzyD8LbAjM9iYoG2LSmSTAGGFfed4I9/pub?gid=0&single=true&output=csv"

s=requests.get(url).content
df=pd.read_csv(StringIO(s.decode('utf-8')))
df.head()

,Level,Title,Text,Source,Type
0,A1,Meine Familie,Zu meiner Familie gehören vier Personen. Die M...,https://german.net/reading/familie/,Reading
1,A1,Maria und ihre Familie,Mein Name ist Maria. Ich bin 30 Jahre alt. Zu ...,https://german.net/reading/marias-familie/,Reading
2,A1,Ich bin Tom,Hallo! Ich bin Tom Maier. Ich bin 13 Jahre alt...,https://german.net/reading/tom/,Reading
3,A1,Freundinnen,Ricarda ist 21 Jahre alt und wohnt in Lübeck. ...,https://german.net/reading/freundinnen/,Reading
4,A1,Einkaufen im Supermarkt,Frau Meier geht heute in den Supermarkt. Ihr M...,https://german.net/reading/einkaufen/,Reading


In [2]:
#clean data
import re
def clean(text):
    text = re.sub("Stadt X\,?\.?\s?(den )?\d+((\,)|(\.)|(\-))\d+((\,)|(\.)|(\. )|(\-))\d+ ", "", text)
    text = re.sub("^.{10,250}Sehr gee?h?e?e?rte?( Damen )?((und)|(oder))? He?er?ren", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,100}Sehr Geherte mein Damen und Herren", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,100}Liebe Dana", "Liebe Dana", text)
    text = re.sub("^.{10,100}Lieber? Silke", "Liebe Silke", text)
    text = re.sub("^.{10,100}Liebe Maria", "Liebe Maria", text)
    text = re.sub("^.{10,100}Liebe Katharina", "Liebe Katharina", text)
    text = re.sub("(\S+)\.(\S+)", r"\1. \2", text)
    text = re.sub("www. ", r"www.", text)
    text = re.sub("^.{10,100}Lieber Michael", "Lieber Michael", text)
    text = re.sub("^.{10,100}Lieber Ingo", "Lieber Ingo", text)
    text = re.sub("^.{10,100}Lieber Julia", "Lieber Julia", text)
    text = re.sub("^.{10,100}Lieber Eva", "Lieber Eva", text)
    text = re.sub("^.{10,100}Hallo Eva", "Hallo Eva", text)
    text = re.sub("^.{10,100}Liebe Julia", "Liebe Julia", text)
    text = re.sub("^.{10,100}Hallo Julia", "Hallo Julia", text)
    text = re.sub("^.{10,200}Sehr geerhte Damen und Herren", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,200}Sehr Geehrte Damen und Herren", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,200}Sehr geehrte Damen und Herrn", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,200}See?hr gerrhte Herren und Damen", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,200}Sehr geehte Damen und Herr", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,200}Seehr geehrte Damen und Herren", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,200}Seher geherte Damen und Herren", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,200}Sehr geherte Herrn", "Sehr geehrte Herren", text)
    text = re.sub("^.{10,200}Sehr geehrte Fir", "Sehr geehrte Fir", text)
    text = re.sub("^.{10,200}Sehr geehrte Damen u. Herren", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,200}Sehr geehrte Damen u Herren", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,200}Seehr Geehrte Damen und He", "Sehr geehrte Damen und He", text)
    text = re.sub("^.{10,200}Sehr geehrte Au-", "Sehr geehrte Au-", text)
    return text
    


In [3]:
df.head()

,Level,Title,Text,Source,Type
0,A1,Meine Familie,Zu meiner Familie gehören vier Personen. Die M...,https://german.net/reading/familie/,Reading
1,A1,Maria und ihre Familie,Mein Name ist Maria. Ich bin 30 Jahre alt. Zu ...,https://german.net/reading/marias-familie/,Reading
2,A1,Ich bin Tom,Hallo! Ich bin Tom Maier. Ich bin 13 Jahre alt...,https://german.net/reading/tom/,Reading
3,A1,Freundinnen,Ricarda ist 21 Jahre alt und wohnt in Lübeck. ...,https://german.net/reading/freundinnen/,Reading
4,A1,Einkaufen im Supermarkt,Frau Meier geht heute in den Supermarkt. Ihr M...,https://german.net/reading/einkaufen/,Reading


In [4]:
import urllib
url = "http://www.cis.uni-muenchen.de/~schmid/tools/RFTagger/data/RFTagger.tar.gz"
filename = "RFTagger.tar.gz"
urllib.request.urlretrieve(url, filename)
import tarfile
tar = tarfile.open(filename)
tar.extractall()
tar.close()
from subprocess import run
run(["make"], cwd="RFTagger/src")

CompletedProcess(args=['make'], returncode=0)

In [5]:
from subprocess import check_output
from nltk.tokenize import sent_tokenize, word_tokenize

def preprocessing(text):
    file = open("RFTagger/temp.txt", "w")
    file.write("\n\n".join("\n".join(word_tokenize(sentence, language='german')) for sentence in sent_tokenize(text, language='german')))
    file.close()
    test_tagged = check_output(["src/rft-annotate", "lib/german.par", "temp.txt"], cwd="RFTagger").decode("utf-8").split("\n")
    
    sent_indecies = [index for index in range(len(test_tagged)) if test_tagged[index] == '']
    sent_indecies = [-1] + sent_indecies

    test_tagged_saparated= []
    for entry in test_tagged:
        temp = entry.split("\t")
        if len(temp) == 2:
            test_tagged_saparated.append([temp[0]] + temp[1].split("."))# + [temp[2]])
        else:
            test_tagged_saparated.append([])

    output = []
    for start, end in zip(sent_indecies, sent_indecies[1:]):
        output.append(test_tagged_saparated[start+1: end])

    return output

In [6]:
#!pip install standordcorenlp
#!apt install openjdk-11-jre

#download Stanford Parser
import urllib
url = "http://nlp.stanford.edu/software/stanford-corenlp-full-2018-02-27.zip"
filename = "stanford-corenlp-full-2018-02-27.zip"
urllib.request.urlretrieve(url, filename)
import zipfile
zip_obj = zipfile.ZipFile(filename)
zip_obj.extractall()
zip_obj.close()

url = "http://nlp.stanford.edu/software/stanford-german-corenlp-2018-02-27-models.jar"
filename = "stanford-german-corenlp-2018-02-27-models.jar"
urllib.request.urlretrieve(url, filename)

import shutil
shutil.move("./stanford-german-corenlp-2018-02-27-models.jar", "./stanford-corenlp-full-2018-02-27/stanford-german-corenlp-2018-02-27-models.jar")

'./stanford-corenlp-full-2018-02-27/stanford-german-corenlp-2018-02-27-models.jar'

In [7]:
#stanford parser

from stanfordcorenlp import StanfordCoreNLP
from nltk.tokenize import sent_tokenize
import re
import sys

    
def parsing(text):
    output=[]
    text = sent_tokenize(text)
    trees=[]
    
    for sentence in text:
        try:
            tree = re.sub(" +", " ", nlp.parse(sentence).replace("\n", ""))
            output.append(tree)
        except ValueError:
            output.append("")
        
    return output

In [8]:
from tqdm import tqdm
def executer(df, function, part_of_df, new_column):
    result = []
    for entry in tqdm(df[part_of_df]):
        result.append(function(entry))
    df[new_column]=result

In [9]:
executer(df, clean, "Text", "cleanedText")
executer(df, preprocessing, "cleanedText", "RFTagger")
nlp = StanfordCoreNLP(r'./stanford-corenlp-full-2018-02-27', lang='de', memory='4g')
executer(df, parsing, "cleanedText", "parsedText")
nlp.close()  # Do not forget to close! The backend server will consume a lot memory.

100%|██████████| 1138/1138 [18:48<00:00,  1.01it/s]


In [10]:
def save_df(path,df):
    df.to_csv(path,sep="|",index=False)

In [11]:
display(df.head(n=100))
filename = "preprocessed_text_df.csv"
save_df(filename, df)

,Level,Title,Text,Source,Type,cleanedText,RFTagger,parsedText
0,A1,Meine Familie,Zu meiner Familie gehören vier Personen. Die M...,https://german.net/reading/familie/,Reading,Zu meiner Familie gehören vier Personen. Die M...,"[[[Zu, APPR, Dat], [meiner, PRO, Poss, Attr, -...",[(ROOT (S (PP (APPR Zu) (PPOSAT meiner) (NN Fa...
1,A1,Maria und ihre Familie,Mein Name ist Maria. Ich bin 30 Jahre alt. Zu ...,https://german.net/reading/marias-familie/,Reading,Mein Name ist Maria. Ich bin 30 Jahre alt. Zu ...,"[[[Mein, PRO, Poss, Attr, -, Nom, Sg, Masc], [...",[(ROOT (S (NP (PPOSAT Mein) (NN Name)) (VAFIN ...
2,A1,Ich bin Tom,Hallo! Ich bin Tom Maier. Ich bin 13 Jahre alt...,https://german.net/reading/tom/,Reading,Hallo! Ich bin Tom Maier. Ich bin 13 Jahre alt...,"[[[Hallo, ITJ], [!, SYM, Pun, Sent]], [[Ich, P...","[(ROOT (NUR (ITJ Hallo) ($. !))), (ROOT (S (PP..."
3,A1,Freundinnen,Ricarda ist 21 Jahre alt und wohnt in Lübeck. ...,https://german.net/reading/freundinnen/,Reading,Ricarda ist 21 Jahre alt und wohnt in Lübeck. ...,"[[[Ricarda, N, Name, Nom, Sg, Fem], [ist, VFIN...",[(ROOT (CS (S (NE Ricarda) (VAFIN ist) (AP (NP...
4,A1,Einkaufen im Supermarkt,Frau Meier geht heute in den Supermarkt. Ihr M...,https://german.net/reading/einkaufen/,Reading,Frau Meier geht heute in den Supermarkt. Ihr M...,"[[[Frau, N, Reg, Nom, Sg, Fem], [Meier, N, Nam...",[(ROOT (S (NP (NN Frau) (NE Meier)) (VVFIN geh...
5,A1,Frühstück,Es ist 7 Uhr am Morgen. Familie Müller sitzt a...,https://german.net/reading/fruehstueck/,Reading,Es ist 7 Uhr am Morgen. Familie Müller sitzt a...,"[[[Es, PRO, Pers, Subst, 3, Nom, Sg, Neut], [i...",[(ROOT (S (PPER Es) (VAFIN ist) (NP (CARD 7) (...
6,A1,Susanne schreibt einen Brief,Lieber Thomas! Jetzt bist du weit weg! Ich bin...,https://german.net/reading/brief/,Reading,Lieber Thomas! Jetzt bist du weit weg! Ich bin...,"[[[Lieber, N, Reg, Nom, Sg, Masc], [Thomas, N,...",[(ROOT (S (ADJD Lieber) (NP (NE Thomas)) ($. !...
7,A1,Der Skiurlaub,Hartmut hat im Februar eine Reise gemacht: Er ...,https://german.net/reading/skiurlaub/,Reading,Hartmut hat im Februar eine Reise gemacht: Er ...,"[[[Hartmut, N, Name, Nom, Sg, Masc], [hat, VFI...",[(ROOT (CS (S (NE Hartmut) (VAFIN hat) (VP (PP...
8,A1,Struppi,Jan hat einen kleinen Hund. Er heißt Struppi u...,https://german.net/reading/struppi/,Reading,Jan hat einen kleinen Hund. Er heißt Struppi u...,"[[[Jan, N, Name, Nom, Sg, Masc], [hat, VFIN, H...",[(ROOT (S (NE Jan) (VAFIN hat) (NP (ART einen)...
9,A1,In der Schule,Heute ist der erste Schultag. Lena steht mit i...,https://german.net/reading/schule/,Reading,Heute ist der erste Schultag. Lena steht mit i...,"[[[Heute, ADV], [ist, VFIN, Sein, 3, Sg, Pres,...",[(ROOT (S (ADV Heute) (VAFIN ist) (NP (ART der...


In [12]:
###Create Sentence Dataset###
levelsText = []
sentences = []
for text, level in tqdm(zip(df["cleanedText"], df["Level"])):
    for sentence in sent_tokenize(text):
        levelsText.append(level)
        sentences.append(sentence)

sentence_df= pd.DataFrame()
sentence_df["sentences"] = sentences
sentence_df["levelsText"] = levelsText
sentence_df.head()

1138it [00:00, 2802.07it/s]


,sentences,levelsText
0,Zu meiner Familie gehören vier Personen.,A1
1,Die Mutter bin ich und dann gehört natürlich m...,A1
2,"Wir haben zwei Kinder, einen Sohn, der sechs J...",A1
3,Wir wohnen in einem kleinen Haus mit einem Gar...,A1
4,Dort können die Kinder ein bisschen spielen.,A1


In [13]:
executer(sentence_df, preprocessing, "sentences", "RFTagger")
nlp = StanfordCoreNLP(r'./stanford-corenlp-full-2018-02-27', lang='de', memory='4g')
executer(sentence_df, parsing, "sentences", "parsedText")
nlp.close()  # Do not forget to close! The backend server will consume a lot memory.


 67%|██████▋   | 8605/12846 [64:31:10<31:47:55, 26.99s/it]

KeyboardInterrupt: 

In [ ]:
display(sentence_df.head(n=100))
filename = "preprocessed_sentence_df.csv"
save_df(filename, sentence_df)